In [2]:
import tensorflow as tf 
import tensorflow.keras as keras

In [3]:
(train_data,train_label),(test_data,test_label) = keras.datasets.fashion_mnist.load_data()

In [4]:
print(train_data.shape,train_label.shape)

(60000, 28, 28) (60000,)


In [12]:
def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)/255.
    y = tf.cast(y,dtype=tf.int32)
    return x,y

In [14]:
batchsize = 128

db = tf.data.Dataset.from_tensor_slices((train_data,train_label))
db = db.map(preprocess).shuffle(10000).batch(batchsize)


test_db = tf.data.Dataset.from_tensor_slices((test_data,test_label))
test_db = test_db.map(preprocess).batch(batchsize)

In [44]:
model = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(256,activation = tf.nn.relu),
    keras.layers.Dense(128,activation = tf.nn.relu),
    keras.layers.Dense(64,activation = tf.nn.relu),
    keras.layers.Dense(32,activation = tf.nn.relu),
    keras.layers.Dense(10),
])

In [45]:
model.build(input_shape=[None,28,28])

In [46]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          multiple                  0         
_________________________________________________________________
dense_22 (Dense)             multiple                  200960    
_________________________________________________________________
dense_23 (Dense)             multiple                  32896     
_________________________________________________________________
dense_24 (Dense)             multiple                  8256      
_________________________________________________________________
dense_25 (Dense)             multiple                  2080      
_________________________________________________________________
dense_26 (Dense)             multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
________________________________________________

 # !!!!!!注意, 这里我们使用网络层中不使用softmax, 而是在求解损失函数的时候再单独加上直接从logits计算的选项from_logits = True
            # 这样得到的结果更好, 更加收敛

In [49]:
optimizer = tf.optimizers.Adam()

for epoch in range(30):
    for step,(x,y) in enumerate(db):
        y_onehot = tf.one_hot(y,depth = 10)
        with tf.GradientTape() as tape:
            logits = model(x)
            # !!!!!!注意, 这里我们使用网络层中不适用softmax, 而是在求解损失函数的时候再单独加上直接从logits计算的选项from_logits = True
            # 这样得到的结果更好, 更加收敛
            loss_mse = tf.reduce_mean(tf.losses.MSE(logits,y_onehot))
            loss_ce = tf.reduce_mean(tf.losses.categorical_crossentropy(y_onehot,logits,from_logits = True))
        grads = tape.gradient(loss_ce,model.trainable_variables)
        # 梯度计算
        optimizer.apply_gradients(zip(grads,model.trainable_variables))
        # 利用优化器进行梯度更新
        if step % 100 == 0:
            print(epoch,step,'loss_ce: ',loss_ce.numpy(),'loss_mse: ',loss_mse.numpy())
    
    # test
    correct_total = 0
    test_total = 0
    for (x,y) in test_db:
        pred = tf.argmax(model(x),axis=1)
        correct = tf.cast(tf.equal(tf.cast(pred,dtype = tf.int32),y),dtype=tf.int32)
        correct_num = tf.reduce_sum(correct)
        test_total += x.shape[0]
        correct_total += correct_num.numpy()
    accuracy = correct_total/test_total
    print(epoch,'Accuracy: ',accuracy)

0 0 loss_ce:  2.331283 loss_mse:  0.11971407
0 100 loss_ce:  0.53743386 loss_mse:  15.88255
0 200 loss_ce:  0.41462094 loss_mse:  15.989647
0 300 loss_ce:  0.5570139 loss_mse:  20.104881
0 400 loss_ce:  0.44796792 loss_mse:  19.97681
0 Accuracy:  0.8434
1 0 loss_ce:  0.53792655 loss_mse:  19.671814
1 100 loss_ce:  0.33905968 loss_mse:  19.170256
1 200 loss_ce:  0.4505527 loss_mse:  21.222225
1 300 loss_ce:  0.29551846 loss_mse:  23.686775
1 400 loss_ce:  0.42928818 loss_mse:  21.344746
1 Accuracy:  0.8568
2 0 loss_ce:  0.31791908 loss_mse:  20.902279
2 100 loss_ce:  0.2940216 loss_mse:  20.540009
2 200 loss_ce:  0.30905962 loss_mse:  28.111431
2 300 loss_ce:  0.29391754 loss_mse:  26.475342
2 400 loss_ce:  0.45076835 loss_mse:  24.39349
2 Accuracy:  0.8703
3 0 loss_ce:  0.3147937 loss_mse:  24.587744
3 100 loss_ce:  0.33674136 loss_mse:  25.551174
3 200 loss_ce:  0.17783052 loss_mse:  30.416853
3 300 loss_ce:  0.31779572 loss_mse:  29.48692
3 400 loss_ce:  0.26135582 loss_mse:  30.3595

mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
#因为每个样本都是28*28个灰度（2的八次方）图片，其可以用0-255进行标示。
#因此单个样本28*28就有768个像素点，为了便于计算，使得计算能够快速收敛，
#把用每个像素点的值除以255，让其映射到0-1的区间，这个过程叫归一化（Normalization）。
training_images=training_images / 255.0
test_images=test_images / 255.0
#定义模型，模型是序列的特性的，因此选用Sequential来定义，且具有三个层
model = tf.keras.models.Sequential([
#第一层，Flatten层，其作用是将每个样本从28*28的格式，按一定的顺序变成一个1-D的含有768个
#元素的数组（向量）
  tf.keras.layers.Flatten(),
#第二层，Dense是全连接层，他具有128个神经元（即输入输出口径有128个，激活函数是relu）
#激活函数relu表示从输出到输出的过程是非线性的，而且relu是一种简单粗暴的方式，
#其结果就是，当计算值小于0时，强制让其等于0；当计算结果大于0时，就取计算结果。
#从某种程度上讲提高了计算速度，保证了稀疏性。
  tf.keras.layers.Dense(128, activation=tf.nn.relu),
#第三层，Dense是全连接层，他具有10个神经元，因为有10类的label,其激活函数softmax的
#作用就是计算每个样本数据data与属于这个10个label的概率，结果去概率值最大的label,作为最终结果。
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
#配置模型，使用adam优化，损失函数是sparse_categorical_crossentropy，使用精确性来度
#量模型优劣
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#开始训练，给模型喂训练数据集（training_images, training_labels）
#epochs=5, 意思是用这个训练集将模型训练5遍。
model.fit(training_images, training_labels, epochs=5)

#使用测试集数据来验证模型，查看模型的好坏
test_loss = model.evaluate(test_images, test_labels)